Подключаем библиотеки

In [ ]:
from nornir import InitNornir
from nornir_netmiko.tasks import netmiko_send_command, netmiko_send_config
from nornir_jinja2.plugins.tasks import template_file
from nornir_utils.plugins.functions import print_result
from nornir.core.task import Task, Result
import pynetbox

Инициализируем nornir

In [ ]:
nr = InitNornir(
    inventory={
        "plugin": "NetBoxInventory2",
        "options": {
            "nb_url": "http://10.30.1.226:8000",
            "nb_token": "dc5b766afda60e647a9fcc384c736a9689da8618",
            "group_file": "./inventory/groups.yml",
            "defaults_file": "./inventory/defaults.yml",
        },
    },
)

Инициализируем pynetbox

In [ ]:
nb_url = nr.config.inventory.options['nb_url']
nb_token = nr.config.inventory.options['nb_token']
nb = pynetbox.api(
    nb_url,
    token=nb_token
)

In [ ]:
devices = nb.dcim.devices.all()
for device in devices:
    print(device)

In [191]:
#device_type = nb.dcim.device_types.get(slug='cisco_ios').id
#device_type
devices = nb.dcim.devices.filter(
    role='access_switch',
    site = 'plgn',
    device_type = 'cisco_ios'
)
""" for device in devices:
    print(device) """

' for device in devices:\n    print(device) '

In [192]:
host = ""
for host in devices:
    #print(host)
    #print(nr.inventory.hosts[host].username)
    print(nr.inventory.hosts[host.name].username)
#print(nr.inventory.hosts[].username)

sa
sa
